In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')

NameError: name 'dbop' is not defined

In [5]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')

select * from stock_daily_basic where date>=20100101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(2906245, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2906245 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-10 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 188.6 MB


In [6]:
df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [7]:
df = df.join(df_d_basic.drop(columns='close'),how='inner')

In [39]:
# index = df_d.index[(df_d['close/120ma']<=0.69)
#                  & (df_d['close/30ma']<=0.778)
#                  & (df_d['close/60ma']<=0.65)
#                  & (df_d['close/60ma']>=0.55)
#                 ]
# print(df['turnover_rate_f'].head())
# df['p10mean_turnover_rate_f'] = df.reset_index('code').groupby('code')['turnover_rate_f'].rolling(10).mean()
stock_mask = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close0']>5) & (df['p10mean_turnover_rate_f']<5) & (df['amt']>0.5)
print(stock_mask.sum(),stock_mask.mean())
# print(len(index))

874433 0.30107690151775734


In [76]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [42]:
features = []
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
result_50 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=50)
result_100 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=100)

close/10ma 
                     mean    median
bin                               
(0.594, 0.908]  0.001781 -0.000923
(0.908, 0.929]  0.001436 -0.004909
(0.929, 0.941]  0.001023 -0.006325
(0.941, 0.949]  0.001051 -0.008065
(0.949, 0.955]  0.000276 -0.009680
(0.955, 0.96]   0.001603 -0.009234
(0.96, 0.964]   0.000323 -0.010316
(0.964, 0.968]  0.000398 -0.010433
(0.968, 0.971] -0.000413 -0.011660
(0.971, 0.974] -0.000824 -0.012613
(0.974, 0.976] -0.000370 -0.011739
(0.976, 0.979] -0.000705 -0.012586
(0.979, 0.981] -0.000639 -0.012265
(0.981, 0.983] -0.000778 -0.012445
(0.983, 0.985] -0.001319 -0.013158
(0.985, 0.987] -0.000381 -0.012180
(0.987, 0.988] -0.001634 -0.012780
(0.988, 0.99]  -0.000948 -0.012539
(0.99, 0.992]  -0.001006 -0.012451
(0.992, 0.993] -0.000211 -0.011628
(0.993, 0.995] -0.000806 -0.012441
(0.995, 0.997] -0.000375 -0.011613
(0.997, 0.998]  0.000033 -0.011297
(0.998, 1.0]   -0.000131 -0.011406
(1.0, 1.002]   -0.000098 -0.011414
(1.002, 1.003]  0.000216 -0.011583
(1.003,

                     mean    median
bin                               
(0.799, 0.938]  0.001977 -0.002706
(0.938, 0.953]  0.001638 -0.005647
(0.953, 0.961]  0.002943 -0.006258
(0.961, 0.967]  0.002095 -0.008132
(0.967, 0.971]  0.001947 -0.008836
(0.971, 0.974]  0.000602 -0.010181
(0.974, 0.976]  0.000872 -0.009917
(0.976, 0.979]  0.000219 -0.011611
(0.979, 0.981] -0.000018 -0.011389
(0.981, 0.983]  0.000414 -0.011154
(0.983, 0.984]  0.000030 -0.011584
(0.984, 0.986] -0.000366 -0.011848
(0.986, 0.987] -0.000885 -0.012007
(0.987, 0.989] -0.001514 -0.012597
(0.989, 0.99]  -0.001137 -0.012574
(0.99, 0.991]  -0.001487 -0.012607
(0.991, 0.992] -0.000509 -0.011935
(0.992, 0.993] -0.001162 -0.012786
(0.993, 0.995] -0.000963 -0.011550
(0.995, 0.996] -0.000889 -0.011966
(0.996, 0.997] -0.000122 -0.011074
(0.997, 0.998] -0.000257 -0.010807
(0.998, 0.999]  0.000298 -0.011097
(0.999, 1.0]    0.000269 -0.010238
(1.0, 1.001]   -0.000399 -0.011317
(1.001, 1.002]  0.000472 -0.010851
(1.002, 1.003]  0.0

close/20ma 
                     mean    median
bin                               
(0.53, 0.841]   0.003919  0.003620
(0.841, 0.87]   0.001476 -0.001604
(0.87, 0.886]   0.000548 -0.004583
(0.886, 0.898]  0.001192 -0.005727
(0.898, 0.907]  0.000240 -0.007009
(0.907, 0.914]  0.000291 -0.007984
(0.914, 0.92]   0.000036 -0.008026
(0.92, 0.925]  -0.000262 -0.008513
(0.925, 0.929] -0.000751 -0.009317
(0.929, 0.933] -0.000143 -0.009567
(0.933, 0.937] -0.001085 -0.011725
(0.937, 0.94]  -0.000099 -0.010920
(0.94, 0.943]  -0.000216 -0.009913
(0.943, 0.946]  0.000728 -0.010714
(0.946, 0.949] -0.001138 -0.011721
(0.949, 0.951] -0.000611 -0.011992
(0.951, 0.953] -0.000673 -0.011589
(0.953, 0.956] -0.000438 -0.012295
(0.956, 0.958] -0.000432 -0.012500
(0.958, 0.96]  -0.000757 -0.012139
(0.96, 0.962]  -0.001154 -0.013477
(0.962, 0.964] -0.000081 -0.012510
(0.964, 0.965] -0.000784 -0.011210
(0.965, 0.967] -0.000377 -0.012162
(0.967, 0.969] -0.000666 -0.011628
(0.969, 0.97]  -0.000868 -0.013066
(0.97, 

close/5ma 
                       mean    median
bin                                 
(0.8001, 0.9215]  0.003975  0.000658
(0.9215, 0.9382] -0.000020 -0.004944
(0.9382, 0.9472]  0.001885 -0.005195
(0.9472, 0.9532]  0.001391 -0.005966
(0.9532, 0.9577]  0.002863 -0.006194
(0.9577, 0.9612]  0.003024 -0.006289
(0.9612, 0.9641]  0.001140 -0.008469
(0.9641, 0.9665]  0.003049 -0.007677
(0.9665, 0.9687]  0.002725 -0.007383
(0.9687, 0.9706]  0.001169 -0.009917
(0.9706, 0.9722]  0.000557 -0.010178
(0.9722, 0.9738]  0.000646 -0.010190
(0.9738, 0.9752]  0.002053 -0.009013
(0.9752, 0.9765] -0.000309 -0.010959
(0.9765, 0.9777]  0.000771 -0.010517
(0.9777, 0.9788] -0.000332 -0.012500
(0.9788, 0.9799] -0.000418 -0.012214
(0.9799, 0.9809]  0.000382 -0.010691
(0.9809, 0.9818]  0.001297 -0.010826
(0.9818, 0.9827] -0.000469 -0.011494
(0.9827, 0.9836] -0.000240 -0.012354
(0.9836, 0.9844]  0.000300 -0.010727
(0.9844, 0.9852] -0.000846 -0.011723
(0.9852, 0.9859]  0.000115 -0.011902
(0.9859, 0.9866] -0.001604

In [43]:
print(result.sort_values(['median_std','mean_std'],ascending=False))
print(result_50.sort_values(['median_std','mean_std'],ascending=False))
print(result_100.sort_values(['median_std','mean_std'],ascending=False))


             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.002214    0.002930   0.006737    -0.002907  0.000177   
close/30ma   0.001786    0.002642   0.005013    -0.004687 -0.000075   
close/20ma   0.001518    0.002552   0.004747    -0.005438  0.000216   
close/120ma  0.002525    0.002524   0.006875    -0.003664  0.000394   
close/10ma   0.001263    0.002277   0.004025    -0.005446  0.000436   
close/5ma    0.001634    0.002189   0.005813    -0.005129  0.000456   
close/250ma  0.001411    0.001058   0.005713    -0.006659  0.001670   

             median_q4%  
close/60ma    -0.011214  
close/30ma    -0.011815  
close/20ma    -0.011823  
close/120ma   -0.010571  
close/10ma    -0.011761  
close/5ma     -0.011686  
close/250ma   -0.009766  
             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.001779    0.002629   0.003023    -0.005295 -0.001769   
close/30ma   0.001455    0.002506   0.001915    -0.006860 -0.001968   
close/120m

In [50]:
# strategy_mask = (
#     (df['close/120ma']<0.7) 
#     & (df['close/60ma']<0.75) & (df['close/60ma']>0.55) 
#     & (df['close/30ma']<0.79) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
#                 )
strategy_mask = (
    (df['close/120ma']<0.72) 
    & (df['close/60ma']<0.77) 
    & (df['close/60ma']>0.55) 
    & (df['close/30ma']<0.82) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
                )
print(strategy_mask.sum())

8281


In [90]:
print(df_r.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
print(df_r2.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
print(df_r2.loc[df.index[stock_mask & strategy_mask],'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))
df_r2.loc[df.index[stock_mask & strategy_mask],:].sort_values('date',ascending=False)[:50]

mean         0.026368
median       0.022989
max          0.655405
min         -0.207692
std          0.069571
size      1728.000000
Name: r, dtype: float64
mean         0.034785
median       0.015994
max          0.655405
min         -0.492201
std          0.093027
size      1728.000000
Name: r, dtype: float64
                mean    median       max       min       std  size
date                                                              
2011-12-31 -0.018218 -0.022790  0.153846 -0.103537  0.061254    20
2012-03-31  0.145057  0.145854  0.246291  0.040000  0.072987     5
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.179314  0.175102  0.203595  0.149301  0.021886     5
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30 -0.025308 -0.027143  0.074227 -0.082334  0.031717    54
2013-09-30 -0.006492 -0.011556  0.087558 -0.071948  0.045435    34
2013-12-31  0.0723

buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002583.SZ 2020-04-03   33.788223   36.149460   2103.0  2100       True   
          2020-04-02   34.013103   36.149460   2103.0  2099       True   
          2020-04-01   32.832481   36.149460   2103.0  2098       True   
002123.SZ 2020-04-01   97.448639  146.759995   2357.0  2351       True   
601808.SH 2020-04-01   12.939040   15.308160   2488.0  2485       True   
002583.SZ 2020-03-31   32.495163   36.149460   2103.0  2097       True   
002123.SZ 2020-03-31   97.448639  146.759995   2357.0  2350       True   
002583.SZ 2020-03-30   34.181763   36.149460   2103.0  2096       True   
002123.SZ 2020-03-30  101.484535  146.759995   2357.0  2349       True   
601808.SH 2020-03-30   12.802361   15.308160   2488.0  2483       True   
002563.SZ 2020-03-23   32.643448   37.198349   2190.0  2179       True   
002387.SZ 2020-03-23   39.458221   42.553680   2269.0  2265       True   
002008.SZ 2020-03-23  356.953125  378.648529   2459.0  2457       True   
000963.SZ 2020-03-19  203.533234  250.912689   2447.0  2433       True   
002124.SZ 2020-02-05   99.461319  119.375961   2398.0  2388       True   
          2019-09-26  114.005714  140.745041   2309.0  2306       True   
600338.SH 2019-08-15   21.907898   24.061869   1998.0  1995       True   
          2019-08-14   21.189911   24.061869   1998.0  1994       True   
601155.SH 2019-08-09   33.848099   37.280098    896.0   887       True   
          2019-08-08   34.863396   37.280098    896.0   886       True   
          2019-08-07   35.263798   37.280098    896.0   885       True   
600884.SH 2019-06-06  184.293884  219.873703   2277.0  2260       True   
          2019-06-05  185.590439  219.873703   2277.0  2259       True   
          2019-06-04  182.071259  219.873703   2277.0  2258       True   
002310.SZ 2019-05-30  232.374435  277.660004   2058.0  2056       True   
          2019-05-24  247.927063  252.958801   2054.0  2052       True   
          2019-05-23  245.639908  252.958801   2054.0  2051       True   
600703.SH 2019-05-20  198.179504  217.788834   2224.0  2222       True   
002358.SZ 2019-05-17   90.819427   93.933716   2033.0  2030       True   
600518.SH 2019-05-14  259.021057  317.379364   2247.0  2244       True   
          2019-05-13  251.389587  289.995850   2246.0  2243       True   
          2019-05-10  259.469971  271.590546   2243.0  2242       True   
002358.SZ 2019-05-09   96.711327   98.394722   2025.0  2024       True   
          2019-05-08   94.859589   98.394722   2025.0  2023       True   
          2019-05-07   94.438744   98.394722   2025.0  2022       True   
          2019-05-06   96.122139   98.394722   2025.0  2021       True   
300133.SZ 2019-02-01  165.961441  204.162918   1900.0  1889       True   
          2019-01-31  164.443497  190.248474   1899.0  1888       True   
600518.SH 2019-01-31  266.652527  330.397766   2183.0  2180       True   
300072.SZ 2019-01-31  149.928513  181.853989   2070.0  2064       True   
600518.SH 2019-01-30  265.754730  330.397766   2183.0  2179       True   
          2019-01-29  262.612335  330.397766   2183.0  2178       True   
          2019-01-28  260.367798  302.116425   2182.0  2177       True   
          2019-01-25  297.178406  298.525146   2177.0  2176       True   
          2019-01-24  305.707703  308.850067   2176.0  2175       True   
          2019-01-23  306.605530  311.992432   2175.0  2174       True   
          2019-01-22  317.379364  319.623901   2174.0  2173       True   
600703.SH 2019-01-21  214.242477  224.881577   2147.0  2146       True   
          2019-01-18  198.596725  224.881577   2147.0  2145       True   
600518.SH 2019-01-18  330.846649  335.335754   2172.0  2171       True   

                         sell_at  sell_date         r  
code      date                                         
002583.SZ 2020-04-03   35.924580        NaT  0.063228  
         

In [53]:
df.loc[stock_mask & strategy_mask,['open','close','close0','amt','pct','adj_factor','close/30ma','close/60ma','close/120ma']].sort_values('date',ascending=False)[:50]

open       close     close0        amt        pct  \
code      date                                                                  
002583.SZ 2020-04-03   34.013103   33.169804   5.900000   1.574771  -2.479339   
          2020-04-02   32.832481   34.013103   6.050000   1.759223   3.418803   
          2020-04-01   32.495163   32.888702   5.850000   2.045960   0.343060   
002123.SZ 2020-04-01   97.448639   97.448639  13.280000   1.551491  -1.043218   
601808.SH 2020-04-01   13.166842   12.859310  11.290000   2.480780  -3.339046   
002583.SZ 2020-03-31   34.181763   32.776260   5.830000   1.897822  -2.995014   
002123.SZ 2020-03-31  101.484535   98.475960  13.420000   1.463604  -1.972234   
002583.SZ 2020-03-30   35.418602   33.788223   6.010000   2.625269  -7.253080   
002123.SZ 2020-03-30  103.465797  100.457214  13.690000   1.830260  -3.794795   
601808.SH 2020-03-30   13.155451   12.813750  11.250000   3.264315  -5.303031   
002563.SZ 2020-03-23   33.098938   32.137348   6.350000   0.655843  -5.082214   
002387.SZ 2020-03-23   39.742863   38.639881  10.860000   1.024126  -5.892551   
002008.SZ 2020-03-23  361.547455  350.189301  27.440001   6.945117  -6.666660   
000963.SZ 2020-03-19  207.750244  202.168884  16.299999   4.221209  -3.891516   
002124.SZ 2020-02-05   94.538597   96.664322   8.640000   5.454072   3.349292   
          2019-09-26  128.662003  115.348282  10.310000   6.405008  -9.956330   
600338.SH 2019-08-15   21.189911   21.852669  11.870000   1.378265   0.423014   
          2019-08-14   22.552250   21.760618  11.820000   1.488058  -1.335567   
601155.SH 2019-08-09   34.863396   33.748001  23.600000   5.379502  -2.559859   
          2019-08-08   35.263798   34.634598  24.219999   4.968066  -1.344198   
          2019-08-07   34.405800   35.106499  24.549999   7.168399   2.419686   
600884.SH 2019-06-06  185.590439  182.997360   9.880000   0.987673  -0.503528   
          2019-06-05  182.071259  183.923462   9.930000   1.625894   1.533759   
          2019-06-04  185.220001  181.145142   9.780000   2.679128  -7.298583   
002310.SZ 2019-05-30  243.810181  231.459579   5.060000   3.175980  -5.420560   
          2019-05-24  245.639908  246.097351   5.380000   1.164091  -0.370371   
          2019-05-23  253.873657  247.012207   5.400000   1.268486  -3.052062   
600703.SH 2019-05-20  204.855011  187.123169   8.970000  15.670564 -10.030090   
002358.SZ 2019-05-17   97.132179   90.903603  10.800000   1.934705  -6.250000   
600518.SH 2019-05-14  251.389587  257.225433   5.730000  12.670181  -0.865048   
          2019-05-13  259.469971  259.469971   5.780000  19.168602  -9.968853   
          2019-05-10  252.736328  288.200226   6.420000  26.087208   2.555907   
002358.SZ 2019-05-09   94.859589   96.122139  11.420000   0.963381   0.087643   
          2019-05-08   94.438744   96.037964  11.410000   1.301822  -0.696260   
          2019-05-07   96.122139   96.711327  11.490000   1.305434   0.877964   
          2019-05-06  104.370796   95.869629  11.390000   2.353368  -9.960472   
300133.SZ 2019-02-01  164.443497  163.684525   6.470000   1.183803  -0.154322   
          2019-01-31  180.634857  163.937515   6.480000   1.615739  -9.999996   
600518.SH 2019-01-31  265.754730  262.163452   5.840000   4.619983  -1.351351   
300072.SZ 2019-01-31  149.120285  148.514099   7.350000   2.992081  -4.792744   
600518.SH 2019-01-30  262.612335  265.754730   5.920000   5.739480   1.718211   
          2019-01-29  260.367798  261.265625   5.820000   6.759272  -6.280190   
          2019-01-28  297.178406  278.773102   6.210000   7.815370  -5.766315   
          2019-01-25  305.707703  295.831696   6.590000   7.009233  -3.795612   
          2019-01-24  306.605530  307.503326   6.850000   4.140827   0.292814   
          2019-01-23  317.379364  306.605530   6.830000   6.548916  -4.073030   
          2019-01-22  325.459747  319.623901   7.120000   6.127024  -3.260875   
600703.SH 2019-01-21  198.596725  210.278885  10.080000  13

In [56]:
df_r[df_r['r'].isnull()].is_selled.unique()

array([False])

In [58]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
print(len(index))

1302


In [89]:
df_r2.loc[index,:].sort_values('date',ascending=False)[:50]

buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002506.SZ 2020-04-10         NaN         NaN      NaN  1874      False   
          2020-04-09   15.392941   15.392941   1874.0  1873      False   
          2020-04-08   15.494880   15.494880   1873.0  1872       True   
          2020-04-07   15.545850   15.596820   1872.0  1871       True   
          2020-04-03   15.494880   15.749730   1871.0  1870       True   
002411.SZ 2020-04-02   32.994720   38.910439   2089.0  2087       True   
002506.SZ 2020-04-02   15.494880   15.851670   1870.0  1869       True   
002411.SZ 2020-04-01   32.161518   38.910439   2089.0  2086       True   
002506.SZ 2020-04-01   15.545850   15.851670   1870.0  1868       True   
300618.SZ 2020-03-31  106.319305  121.387108    752.0   747       True   
002353.SZ 2020-03-31  178.804703  214.239792   2352.0  2345       True   
002506.SZ 2020-03-31   15.392941   16.157492   1868.0  1867       True   
002411.SZ 2020-03-31   32.328159   38.910439   2089.0  2085       True   
002506.SZ 2020-03-30   16.412340   16.718161   1867.0  1866       True   
002353.SZ 2020-03-30  178.397400  207.722992   2348.0  2344       True   
002411.SZ 2020-03-30   34.452820   34.452820   2085.0  2084       True   
300618.SZ 2020-03-30  106.547607  121.387108    752.0   746       True   
002411.SZ 2020-03-27   34.577801   35.994240   2084.0  2083       True   
002506.SZ 2020-03-27   16.973009   17.482710   1866.0  1865       True   
002353.SZ 2020-03-27  178.071564  207.722992   2348.0  2343       True   
002411.SZ 2020-03-26   33.744602   36.494160   2083.0  2082       True   
300618.SZ 2020-03-23  108.807777  115.451309    743.0   741       True   
000662.SZ 2020-02-04   20.908781   24.413721   2099.0  2095       True   
600338.SH 2019-08-12   21.300369   24.061869   1998.0  1992       True   
          2019-08-09   21.852669   24.061869   1998.0  1991       True   
          2019-08-08   22.404970   24.061869   1998.0  1990       True   
          2019-08-07   22.699530   24.061869   1998.0  1989       True   
          2019-08-06   22.091999   23.877769   1989.0  1988       True   
002341.SZ 2019-06-27   41.690880   50.872803   2100.0  2099       True   
300090.SZ 2019-06-11   11.288650   13.692039   1939.0  1936       True   
300156.SZ 2019-06-10   21.785040   25.278002   1792.0  1789       True   
300178.SZ 2019-06-10   25.866741   30.049061   1918.0  1915       True   
300090.SZ 2019-06-10   10.050540   13.692039   1939.0  1935       True   
002247.SZ 2019-06-10   17.255421   20.628660   2171.0  2169       True   
600179.SH 2019-06-10    9.337930   10.242331   2021.0  2019       True   
          2019-06-06    8.998779   10.242331   2021.0  2018       True   
002537.SZ 2019-06-06   46.847153   56.274960   1899.0  1888       True   
600179.SH 2019-06-05    9.564030   10.242331   2021.0  2017       True   
002247.SZ 2019-06-05   18.487949   19.461000   2168.0  2167       True   
300090.SZ 2019-06-05    9.030920   13.692039   1939.0  1933       True   
002247.SZ 2019-06-04   18.747431   19.461000   2168.0  2166       True   
300090.SZ 2019-06-04    8.812430   10.414690   1933.0  1932       True   
002247.SZ 2019-06-03   18.812300   19.461000   2168.0  2165       True   
300090.SZ 2019-06-03    9.759220   10.414690   1933.0  1931       True   
600179.SH 2019-05-31   10.604090   11.304999   2016.0  2014       True   
002470.SZ 2019-05-29   17.545591   17.972490   2004.0  2003       True   
002247.SZ 2019-05-28   19.136650   20.823271   2162.0  2161       True   
002470.SZ 2019-05-28   17.545591   17.972490   2004.0  2002       True   
002247.SZ 2019-05-27   18.877171   20.823271   2162.0  2160       True   
002470.SZ 2019-05-27   17.844419   18.100559   2002.0  2001       True   

                         sell_at  sell_date         r  
code      date                                         
002506.SZ 2020-04-10         NaN        NaT       NaN  
         

In [67]:
df_r[(df_r.is_selled==True) & (df_r.r.isnull())].sort_values('date').head(50)

,,buy_at,max,max_idx,idx,is_selled,sell_at,r
code,date,,,,,,,
000418.SZ,2019-04-30,167.895859,175.670792,2205.0,2203,True,NaN,NaN
002450.SZ,2019-07-03,52.103348,53.277512,1912.0,1910,True,NaN,NaN
600401.SH,2019-07-04,2.830200,3.207560,1392.0,1391,True,NaN,NaN
002477.SZ,2019-10-11,3.269920,3.883030,2016.0,2014,True,NaN,NaN
002143.SZ,2019-11-26,0.895160,0.927130,1984.0,1983,True,NaN,NaN
600240.SH,2020-01-21,3.030120,3.349080,2335.0,2333,True,NaN,NaN
000848.SZ,2020-03-12,145.541504,161.439117,2488.0,2471,True,NaN,NaN
600236.SH,2020-03-12,30.058800,31.209702,2409.0,2395,True,NaN,NaN
600729.SH,2020-03-12,81.136620,87.456543,2317.0,2305,True,NaN,NaN


In [74]:
df_r['curr_date']= df_r.index.get_level_values('date')

In [75]:
df_r.head()

buy_at         max  max_idx  idx  is_selled  \
code      date                                                          
000001.SZ 2010-01-04  852.767456  858.153320      1.0    0       True   
          2010-01-05  834.814453  850.254028      5.0    1       True   
          2010-01-06  822.247375  850.254028      5.0    2       True   
          2010-01-07  807.884949  850.254028      5.0    3       True   
          2010-01-08  843.790955  850.254028      5.0    4       True   

                         sell_date     sell_at         r  curr_date  
code      date                                                       
000001.SZ 2010-01-04  1.262909e+18  807.884949 -0.052632 2010-01-04  
          2010-01-05  1.263254e+18  810.398376 -0.029247 2010-01-05  
          2010-01-06  1.263254e+18  810.398376 -0.014411 2010-01-06  
          2010-01-07  1.263254e+18  810.398376  0.003111 2010-01-07  
          2010-01-08  1.263341e+18  785.264221 -0.069362 2010-01-08

In [78]:
df_r1 = pd.read_parquet(r"database\return_5%_10%_20_8_15% close").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [79]:
df_r2 = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [81]:
print(df_r.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)[:4])
print(df_r2.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)==df_r2.r.agg(agg_operations))

mean       0.002731
median    -0.009649
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean      True
median    True
max       True
min       True
std       True
size      True
Name: r, dtype: bool


In [84]:
print(df_r.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))
print(df_r1.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))

                mean    median       max       min       std  size
date                                                              
2011-09-30  0.076663  0.076663  0.076663  0.076663       NaN     1
2011-12-31 -0.035069 -0.034591  0.081654 -0.134314  0.072813    13
2012-03-31  0.113472  0.092634  0.436782 -0.046875  0.112979    18
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.136266  0.136266  0.146096  0.126437  0.013901     2
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30       NaN       NaN       NaN       NaN       NaN     0
2013-09-30       NaN       NaN       NaN       NaN       NaN     0
2013-12-31       NaN       NaN       NaN       NaN       NaN     0
2014-03-31       NaN       NaN       NaN       NaN       NaN     0
2014-06-30       NaN       NaN       NaN       NaN       NaN     0
2014-09-30       NaN       NaN       NaN       NaN       NaN  

In [85]:
print(df_r.loc[index,'r'].agg(agg_operations))
print(df_r1.loc[index,'r'].agg(agg_operations))

mean         0.064221
median       0.058891
max          1.204987
min         -0.554711
std          0.126345
size      1302.000000
Name: r, dtype: float64
mean         0.073225
median       0.048387
max          1.204987
min         -0.554711
std          0.145928
size      1302.000000
Name: r, dtype: float64


In [122]:
print(df.index.is_lexsorted())
# df.groupby('code').apply(lambda x:np.arange(1,len(x+1))).head()
df['tmp'] = 1
df['idx'] = df.reset_index('code').groupby('code')['tmp'].expanding(1).sum().astype('int')
print(df['idx'].head())
del df['tmp']

KeyError: 'tmp'

In [ ]:
# df_target_not_trash = df[stock_mask & strategy_mask]
print(df_target_not_trash.shape)
for i in range(1,6):
#     df_target_not_trash['idx_diff{}'.format(i)] = df_target_not_trash.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target_not_trash['idx_diff{}'.format(i)]==i).sum())
    
# df_target_not_trash = df_target_not_trash.join(df_r,how='inner',rsuffix='r_')
df_target_not_trash = df_target_not_trash.join(df_r2[['r']],how='inner',rsuffix='2')

# 检查拼表后数据是否完整
print(df_target_not_trash.shape)
print('间隔超跌的样本数：',(df_target_not_trash['idx_diff1']>1).sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target_not_trash['idx_diff1'].isnull().sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1'].isnull()]['idx_diff1'].head())

In [128]:
# df_target = df.loc[index]
# print(df.columns)
# df_target = df_target.join(df_r,how='inner',rsuffix='_r')
# df_target = df_target.join(df_r2,how='inner',rsuffix='_r2')
# print(df_target.shape)
for i in range(1,6):
#     df_target['idx_diff{}'.format(i)] = df_target.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r_r2'].agg(agg_operations),'\n')

for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target.loc[df_target['idx_diff1']>k,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff1']>k,'r_r2'].agg(agg_operations),'\n')

print('\nnull')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r'].agg(agg_operations),'\n')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r_r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target.loc[:,'r'].agg(agg_operations),'\n')
print(df_target.loc[:,'r_r2'].agg(agg_operations),'\n')
    
# df_target = df_target.join(df_r,how='inner',rsuffix='r_')
# 检查拼表后数据是否完整
print(df_target.shape)
print('间隔超跌的样本数：',(df_target['idx_diff1']>1).sum())
print(df_target[df_target['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target['idx_diff1'].isnull().sum())
print(df_target[df_target['idx_diff1'].isnull()]['idx_diff1'].head())

idx_diff1==1: 592
mean        0.050100
median      0.040513
max         0.655405
min        -0.554711
std         0.112235
size      592.000000
Name: r, dtype: float64 

mean        0.054878
median      0.024129
max         0.655405
min        -0.554711
std         0.132836
size      592.000000
Name: r_r2, dtype: float64 

idx_diff2==2: 323
mean        0.030315
median      0.022945
max         0.551485
min        -0.554711
std         0.096181
size      323.000000
Name: r, dtype: float64 

mean        0.032791
median      0.005263
max         0.551485
min        -0.554711
std         0.121943
size      323.000000
Name: r_r2, dtype: float64 

idx_diff3==3: 194
mean        0.012319
median      0.013333
max         0.253676
min        -0.554711
std         0.093841
size      194.000000
Name: r, dtype: float64 

mean        0.010770
median     -0.002691
max         0.253676
min        -0.554711
std         0.113661
size      194.000000
Name: r_r2, dtype: float64 

idx_diff4==4: 131
mean   

In [129]:
for i in range(1,6):
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r2'].agg(agg_operations),'\n')
    
for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r2'].agg(agg_operations),'\n')

print('\nNull')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r'].agg(agg_operations),'\n')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target_not_trash['r'].agg(agg_operations),'\n')
print(df_target_not_trash['r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[df_target_not_trash['idx_diff1']!=1,'r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>5)|(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

df_target_not_trash['1+r'] = df_target_not_trash['r']+1
df_target_not_trash['1+r2'] = df_target_not_trash['r2']+1

df_target_not_trash = df_target_not_trash.sort_values(['date','code'])
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0).tail())
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0).tail())
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[500:510])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[500:510])
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[200:210])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[200:210],'\n')

for k in [10,20,50,100,200]:
    print('\nk={}'.format(k))
    print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[k:k+10])
    print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[k:k+10])


print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[:30])
print(df_target_not_trash[['1+r','1+r2','sell_date']][::-1][:30])

print(df_target_not_trash['r'].reset_index('code')['r'].resample('Q').agg(agg_operations))


mean         0.026368
median       0.022989
max          0.655405
min         -0.207692
std          0.069571
size      1728.000000
Name: r, dtype: float64 

mean         0.034785
median       0.015994
max          0.655405
min         -0.492201
std          0.093027
size      1728.000000
Name: r2, dtype: float64 

idx_diff1==1: 592
mean         0.026680
median       0.022405
max          0.268148
min         -0.171271
std          0.061703
size      1064.000000
Name: r, dtype: float64 

mean         0.036855
median       0.015085
max          0.358641
min         -0.492201
std          0.091062
size      1064.000000
Name: r2, dtype: float64 

idx_diff2==2: 323
mean        0.025853
median      0.020886
max         0.242424
min        -0.171271
std         0.061038
size      721.000000
Name: r, dtype: float64 

mean        0.035423
median      0.011073
max         0.358641
min        -0.492201
std         0.091625
size      721.000000
Name: r2, dtype: float64 

idx_diff3==3: 194
mean   

                mean    median       max       min       std  size
date                                                              
2011-12-31 -0.010072 -0.002491  0.081329 -0.103537  0.057601    20
2012-03-31  0.129921  0.123876  0.149573  0.120000  0.012359     5
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.049252  0.053061  0.076647  0.019380  0.020764     5
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30 -0.001443 -0.001713  0.094919 -0.057143  0.030376    54
2013-09-30  0.023118  0.026345  0.109537 -0.066289  0.052144    34
2013-12-31  0.053341  0.053341  0.055400  0.051282  0.002912     2
2014-03-31  0.051143  0.062480  0.074101  0.005512  0.031619     4
2014-06-30  0.035240  0.035240  0.078145 -0.007664  0.060676     2
2014-09-30       NaN       NaN       NaN       NaN       NaN     0
2014-12-31       NaN       NaN       NaN       NaN       NaN  